In [ ]:
# color feature extraction code
import os
import cv2
import numpy as np
import pandas as pd

def extract_color_features(image_path):
    """
    Extracts the following features from an image:
      - RGB minimum: Rmin, Gmin, Bmin
      - RGB maximum: Rmax, Gmax, Bmax
      - RGB mean: Rmean, Gmean, Bmean
      - RGB standard deviation: Rstd, Gstd, Bstd
      - HSV mean: Hmean, Smean, Vmean (Smean is the mean of the S channel)
      - YCbCr mean: Cbmean, Crmean
      - Grayscale mean: Graymean
    """
    image = cv2.imread(image_path)
    if image is None:
        return None  # Handle unreadable images
    
    # Convert image from BGR (OpenCV default) to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Split RGB channels
    R, G, B = image_rgb[:,:,0], image_rgb[:,:,1], image_rgb[:,:,2]
    
    # Convert to HSV and split channels (including S channel)
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    H, S, V = image_hsv[:,:,0], image_hsv[:,:,1], image_hsv[:,:,2]
    
    # Convert to YCbCr and split channels
    image_ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    _, Cb, Cr = image_ycbcr[:,:,0], image_ycbcr[:,:,1], image_ycbcr[:,:,2]
    
    # Convert to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Compute statistics
    features = {
        "Rmin": np.min(R),   "Gmin": np.min(G),   "Bmin": np.min(B),
        "Rmax": np.max(R),   "Gmax": np.max(G),   "Bmax": np.max(B),
        "Rmean": np.mean(R), "Gmean": np.mean(G), "Bmean": np.mean(B),
        "Rstd": np.std(R),   "Gstd": np.std(G),   "Bstd": np.std(B),
        "Hmean": np.mean(H), "Smean": np.mean(S), "Vmean": np.mean(V),
        "Cbmean": np.mean(Cb), "Crmean": np.mean(Cr),
        "Graymean": np.mean(image_gray)
    }
    
    return features

def process_images(main_folder_path):
    """
    Reads images from all subfolders, extracts features, and saves them to a CSV file in the same directory as the input folder.
    The output CSV will include two extra columns:
      - "Class": the subfolder name in which the image is present.
      - "Image_ID": the image name without its extension.
    """
    data = []
    
    for root, _, files in os.walk(main_folder_path):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                image_path = os.path.join(root, filename)
                features = extract_color_features(image_path)
                if features:
                    # Get the relative path from the main folder
                    relative_path = os.path.relpath(image_path, main_folder_path)
                    # Split the relative path into parts
                    parts = relative_path.split(os.sep)
                    # The first part is assumed to be the subfolder (i.e. "Class")
                    class_name = parts[0] if len(parts) > 1 else "unknown"
                    # Get the image id (the filename without its extension)
                    image_id = os.path.splitext(parts[-1])[0]
                    
                    features["Class"] = class_name
                    features["Image_ID"] = image_id  # Saving image name without extension
                    data.append(features)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Get the parent directory of the input folder
    parent_dir = os.path.dirname(main_folder_path)
    
    # Create the output CSV filename in the same directory as the input folder
    output_csv = os.path.join(parent_dir, "color_features_full.csv")
    
    # Save the DataFrame to CSV
    df.to_csv(output_csv, index=False)
    print(f"Feature extraction complete. Saved to {output_csv}")

#  usage
main_folder_path = r"F:\Optimized_output_final"
process_images(main_folder_path)
